## 구현 순서

1. SageMaker를 이용한 MNIST 모델 훈련
2. SageMaker 훈련모델 배포
3. Neo를 사용한 훈련 모델 배포
4. 추론 컨테이너를 지정한 Neo 훈련 모델 배포


# 1. SageMaker를 이용한 MNIST 모델 훈련

### 환경 설정

In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_region_name

In [2]:
tf_version = '1.15.2'

In [3]:
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/ mnist --recursive

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_labels.npy to mnist/eval_labels.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_labels.npy to mnist/train_labels.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_data.npy to mnist/eval_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_data.npy to mnist/train_data.npy


In [4]:
training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)

In [5]:
# mnist 훈련 스크립트
training_script = "mnist.py"

### SageMaker 훈련 작업 생성

In [6]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    # MNIST 훈련 script
    entry_point=training_script,
    role=role,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    framework_version=tf_version,
    py_version="py3",
)

In [ ]:
# Sagemaker 훈련 작업 시작
estimator.fit(training_data_uri)

In [8]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_data.npy eval_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_labels.npy eval_labels.npy

eval_data = np.load("eval_data.npy")
eval_labels = np.load("eval_labels.npy")

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_data.npy to ./eval_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_labels.npy to ./eval_labels.npy


# 2. SageMaker 훈련모델 배포

In [9]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

### 훈련 데이터 다운로드

## 첫 번째 실험

- Neo Compile 전 훈련 모델 성능 측정

In [10]:
%%time
# %%time : 해당 셀이 실행되는 데 걸리는 시간 측정

# prediction : 모델의 예측값
# label : 데이터의 실제값
# count : 예측이 틀린 횟수

count = 0
for i in range(0, 10000):
    data = eval_data[i]
    predictions = predictor.predict(data)
    prediction = predictions["predictions"][0]["classes"]
    label = eval_labels[i]
#     print("prediction: {}, label: {}, matched: {}".format(prediction, label, prediction == label))
    if (prediction != label):
        count += 1
count = 100 - count/10000
print("Accuracy: {}".format(count))

Accuracy: 99.9705
CPU times: user 22.3 s, sys: 548 ms, total: 22.9 s
Wall time: 2min 20s


### 엔드포인트 삭제

In [11]:
predictor.delete_endpoint()

# 3. Neo를 사용한 훈련 모델 배포

### 모델 컴파일

In [12]:
# output_path 컴파일된 모델이 S3에 저장되는 위치
output_path = "/".join(estimator.output_path.split("/")[:-1])

# Neo compile할 대상 하드웨어 및 프레임워크 선택
optimized_estimator = estimator.compile_model(
    target_instance_family="ml_c5",
    input_shape={"data": [1, 784]},  # Batch size 1, 1 channel, 28*28 image size.
    output_path=output_path,
    framework="tensorflow",
    framework_version="1.15.3",
)

???????????????????????????.....................................!

### 컴파일 된 모델 엔드포인트에 배포

In [13]:
# 모델 최적화 후 실시간으로 예측 요청을 제공하는 Endpoint에 배포
optimized_predictor = optimized_estimator.deploy(
    initial_instance_count=1, instance_type="ml.c5.xlarge"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

## 두 번째 실험

- Neo compile 후 / 추론 컨테이너 이미지 지정 전 훈련 모델 성능 측정

In [14]:
%%time
# %%time : 해당 셀이 실행되는 데 걸리는 시간 측정

# prediction : 모델의 예측값
# label : 데이터의 실제값
# count : 예측이 틀린 횟수

count = 0
for i in range(0, 10000):
    data = eval_data[i].tolist()
    predictions = optimized_predictor.predict({"instances": np.asarray(data).tolist()})
    prediction = predictions["predictions"][0]["classes"]
    label = eval_labels[i]
#     print("prediction: {}, label: {}, matched: {}".format(prediction, label, prediction == label))
    if (prediction != label):
        count += 1
count = 100 - count/10000
print("Accuracy: {}".format(count))

Accuracy: 99.9705
CPU times: user 21.7 s, sys: 577 ms, total: 22.3 s
Wall time: 1min 19s


### 엔드포인트 삭제

In [15]:
optimized_predictor.delete_endpoint()

# 4. 추론 컨테이너를 지정한 Neo 훈련 모델 배포

### 모델 컴파일

In [16]:
# output_path 컴파일된 모델이 S3에 저장되는 위치
output_path = "/".join(estimator.output_path.split("/")[:-1])

# Neo compile할 대상 하드웨어 및 프레임워크 선택
optimized_estimator = estimator.compile_model(
    target_instance_family="ml_c5",
    input_shape={"data": [1, 784]},  # Batch size 1, 1 channel, 28*28 image size.
    output_path=output_path,
    framework="tensorflow",
    framework_version="1.15.3",
)

???????????????????????????................................!

### 추론 컨테이너 이미지 지정

In [17]:
optimized_estimator.image_uri = (
    "301217895009.dkr.ecr.us-west-2.amazonaws.com/sagemaker-inference-tensorflow:1.15.3-cpu-py3"
)

### 컴파일 된 모델 엔드포인트에 배포

In [18]:
optimized_predictor = optimized_estimator.deploy(
    initial_instance_count=1, instance_type="ml.c5.xlarge"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

## 세 번째 실험

- Neo compile 후 / 추론 컨테이너 이미지 지정 후 훈련 모델 성능 측정

In [19]:
%%time
# %%time : 해당 셀이 실행되는 데 걸리는 시간 측정

# prediction : 모델의 예측값
# label : 데이터의 실제값
# count : 예측이 틀린 횟수

count = 0
for i in range(0, 10000):
    data = eval_data[i].tolist()
    predictions = optimized_predictor.predict({"instances": np.asarray(data).tolist()})
    prediction = predictions["predictions"][0]["classes"]
    label = eval_labels[i]
#     print("prediction: {}, label: {}, matched: {}".format(prediction, label, prediction == label))
    if (prediction != label):
        count += 1
count = 100 - count/10000
print("Accuracy: {}".format(count))

Accuracy: 99.9705
CPU times: user 22.3 s, sys: 567 ms, total: 22.9 s
Wall time: 1min 18s


### 엔드포인트 삭제

In [20]:
optimized_predictor.delete_endpoint()